In [1]:
import glob
import json
import os

import anthropic

In [2]:
conversations = []

paths = glob.glob('./res/297.SNS 데이터 고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

target_count = 30
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:           
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

count

30

In [3]:
prompt = f"""You are given a conversation among three users below. Imagine what the users discussed prior to the conversation.
Please write the previous content, strictly following the tone and format of given conversation, with at least 3000 characters.

{conversations[-2]}
"""

In [5]:
print(conversations[-2])

P01: 응응 요즘 너무 개인 피티 받고 싶은데 트레이너가 나한테 집중하는건 또 부담스럽고
P01: 생각 반반 후라이드 양념
P02: 키키 그럼 한 5대1로 알려주는 곳 알아봐
P03: 피티에 5대 1이 있나?
P03: 필테는 있는 거 아는데
P01: 키키 휴 나 필테 5대 1 해봤지
P02: 찾아보면 피티도 있지 않을까
P03: 필테 좋아?
P03: 나도 나중에 돈 많이 벌면 필테 다녀봐야지 키키
P01: 필테 진짜 헬스처럼 곡소리 나와
P01: 안 쓰던 근육 쓰면 느낌이 아주 ...
P02: 엄청 유연해졌겠네
P02: 필테 했으면
P03: 그치그치
P03: 근데 원래 유연해서 더 유연해질 게 남았어?
P01: 유연하면 어느 동작들에선 유리해서 좋은데 거기서 거의 허벅지랑 엉덩이 근육 집중해
P02: 나는 뻣뻣해서 하다 죽겄다
P03: 키키 인정
P03: 넌 뻣뻣해서 하다가 울듯^^
P03: 나도 필테 다니면 곡소리 날듯...
P01: 너넨 무슨 운동이 재밌을 거 같은데?
P02: 음 나는 공으로 하는 운동 좋아하지
P03: 키키 풋살 다니는 이유가 그건가~
P03: 나는 춤 배우고 싶음
P01: 다들 잘 어울리는 거 좋아하네
P01: 춤은 어디 학원 있지 않을까?
P02: 춤 배우는 학원 많지~
P03: 근데 다 시내쪽에 있더라고 ㅠㅠ
P03: 집 주변에 없어
P01: 나중에 여유 있어지면 나랑 같이 다닐래?
P01: 난 보컬 다녀야지
P02: 둘이 같이 다니면 안 심심하고 좋겠네~
P03: 오 뭔가 보컬학원도 재밌겠다
P03: 노래 잘 부르고 싶긴 해
P01: 그치 취미로 하나 개발하는 거 좋을 듯
P02: 둘이 같이 다녀봐 한 번
P03: 오오 진짜 나중에 여유되면 같이 다녀볼래?


In [8]:
from utils import get_anthropicai_key
client = anthropic.Anthropic(api_key=get_anthropicai_key())
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=4096,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

message.usage  # API 호출 비용 대략 50원

Usage(input_tokens=945, output_tokens=965, cache_creation_input_tokens=0, cache_read_input_tokens=0, cache_creation={'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}, service_tier='standard')

In [9]:
message.content[0].text.split('\n\n')

['이 대화를 보면 세 명의 친구들이 운동과 취미 생활에 대해 이야기하고 있는 것 같습니다. 이들은 평소에도 이런 주제로 자주 대화를 나누는 것 같아요.',
 '먼저 P01은 개인 PT를 받고 싶지만 트레이너가 자신에게 너무 집중하는 것이 부담스럽다고 말합니다. 그래서 다른 방법을 찾아보고 싶어 하는 것 같네요. P02는 5대 1 PT를 추천하고, P03는 그런 방식의 PT가 있는지 궁금해 합니다. P01은 필라테스 5대 1 수업을 해봤다고 말하고, P02는 피티에도 그런 방식이 있을 것 같다고 말합니다.',
 '그 후 P03은 필라테스가 좋은지 물어보고, P01은 필라테스가 헬스와 비슷하게 곡소리가 나고 평소 쓰지 않던 근육을 사용해서 느낌이 좋다고 말합니다. P02는 필라테스를 하면 유연성이 많이 늘었을 것 같다고 말하고, P03은 원래 유연했던 사람은 더 유연해질 게 없을 것 같다고 말합니다.',
 'P01은 필라테스에서 허벅지와 엉덩이 근육을 많이 사용한다고 하고, P02는 자신이 뻣뻣해서 필라테스를 하다가 죽을 것 같다고 말합니다. P03은 자신도 필라테스를 하면 곡소리가 날 것 같다고 말합니다.',
 '그 후 P01은 다들 어떤 운동을 좋아하는지 물어보고, P02는 공으로 하는 운동을 좋아한다고 말합니다. P03은 풋살을 하는 이유가 그런 것 같다고 말하고, 자신은 춤을 배우고 싶다고 말합니다. P01은 춤 학원이 많이 있을 것 같다고 말하고, P03은 집 주변에는 없어서 시내에 가야 한다고 말합니다.',
 '마지막으로 P01은 나중에 여유가 생기면 P03과 함께 보컬 학원에 다니고 싶다고 말하고, P02는 둘이 같이 다니면 좋을 것 같다고 말합니다. P03도 보컬 학원이 재미있을 것 같다고 말하며, 노래를 잘 부르고 싶다고 합니다.',
 '이처럼 이 대화에서는 세 명의 친구들이 운동, 취미, 그리고 앞으로의 계획 등에 대해 이야기하고 있습니다. 이들은 서로 관심사가 비슷하고 좋은 친구 관계를 유지하고 있는 것 같아요.']

In [10]:
lst = message.content[0].text.split('\n\n')[1:-1]
for l in lst[:25]:
    print(l)

먼저 P01은 개인 PT를 받고 싶지만 트레이너가 자신에게 너무 집중하는 것이 부담스럽다고 말합니다. 그래서 다른 방법을 찾아보고 싶어 하는 것 같네요. P02는 5대 1 PT를 추천하고, P03는 그런 방식의 PT가 있는지 궁금해 합니다. P01은 필라테스 5대 1 수업을 해봤다고 말하고, P02는 피티에도 그런 방식이 있을 것 같다고 말합니다.
그 후 P03은 필라테스가 좋은지 물어보고, P01은 필라테스가 헬스와 비슷하게 곡소리가 나고 평소 쓰지 않던 근육을 사용해서 느낌이 좋다고 말합니다. P02는 필라테스를 하면 유연성이 많이 늘었을 것 같다고 말하고, P03은 원래 유연했던 사람은 더 유연해질 게 없을 것 같다고 말합니다.
P01은 필라테스에서 허벅지와 엉덩이 근육을 많이 사용한다고 하고, P02는 자신이 뻣뻣해서 필라테스를 하다가 죽을 것 같다고 말합니다. P03은 자신도 필라테스를 하면 곡소리가 날 것 같다고 말합니다.
그 후 P01은 다들 어떤 운동을 좋아하는지 물어보고, P02는 공으로 하는 운동을 좋아한다고 말합니다. P03은 풋살을 하는 이유가 그런 것 같다고 말하고, 자신은 춤을 배우고 싶다고 말합니다. P01은 춤 학원이 많이 있을 것 같다고 말하고, P03은 집 주변에는 없어서 시내에 가야 한다고 말합니다.
마지막으로 P01은 나중에 여유가 생기면 P03과 함께 보컬 학원에 다니고 싶다고 말하고, P02는 둘이 같이 다니면 좋을 것 같다고 말합니다. P03도 보컬 학원이 재미있을 것 같다고 말하며, 노래를 잘 부르고 싶다고 합니다.


In [11]:
for l in lst[25:50]:
    print(l)

In [ ]:
print(len(message.content[0].text.split('\n\n')[1]))

2418


In [ ]:
lst = message.content[0].text.split('\n\n')[1:-1]
lst

['P02: 안녕~ 오늘 날씨 진짜 좋다~ 너희들 뭐하고 있어?',
 'P03: 나는 지금 집에서 쉬고 있어~ 날씨가 좋아서 창문 열어두고 있는데 기분이 너무 좋아~',
 'P01: 나도 집이야! 오늘 같은 날 밖에 나가고 싶은데 할 일이 너무 많아서 ㅠㅠ',
 'P02: 아 그래? 나는 지금 카페에 와있어~ 커피 마시면서 책 읽고 있는데 너무 좋다~',
 'P03: 와~ 부럽다~ 나도 카페 가고 싶은데... 시험 준비 때문에 집에만 있어야 해서 ㅠㅠ',
 'P01: 맞아 맞아~ 나도 과제랑 시험 준비 때문에 집에만 있어~ 근데 카페에서 공부하면 집중 잘 돼?',
 'P02: 응! 나는 오히려 카페에서 공부하는 게 더 집중이 잘 돼~ 집에 있으면 자꾸 누워있게 되더라고~',
 'P03: 나도 그래~ 집에 있으면 자꾸 침대로 가게 돼서 공부가 안 돼 ㅋㅋㅋ',
 'P01: 아~ 나는 반대야~ 카페에 가면 사람들 구경하느라 집중이 안 돼서 꼭 집에서 공부해~',
 'P02: 그렇구나~ 역시 사람마다 다르네~ 근데 너희 요즘 뭐 재밌는 거 없어? 나 심심해 죽겠어~',
 "P03: 음... 나는 요즘 넷플릭스로 드라마 보고 있어~ '오징어 게임' 봤어?",
 'P01: 아 나도 그거 봤어! 진짜 재밌더라~ 근데 좀 잔인해서 보다가 눈 감은 적도 있어 ㅋㅋㅋ',
 'P02: 나는 아직 못 봤어~ 다들 재밌다고 하던데 나도 한번 봐야겠다~',
 'P03: 응 꼭 봐봐~ 진짜 재밌어! 근데 잔인한 장면이 많아서 밤에 보면 무서울 수도 있어~',
 'P01: 맞아~ 나는 낮에 봤는데도 무서웠어 ㅋㅋㅋ',
 'P02: 그래? 그럼 나는 꼭 낮에 볼게~ 근데 너희 영화는 안 봐?',
 "P03: 나는 영화도 좋아해~ 최근에 본 영화 중에 '듄'이라고 있는데 그거 진짜 좋더라~",
 'P01: 아 나도 그거 보고 싶었는데! 어땠어? 재밌었어?',
 'P03: 응! 진짜 재밌었어~ 영상미도 너무 좋고 스토리도 탄탄해~ 근데 좀 어려울 수 있어~',
 'P02: 나는 그런 SF 영

In [12]:
import pickle

with open('./res/conv_long.pickle', 'wb') as f:
    pickle.dump(lst, f)